In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch-forecasting
!pip install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 35.5 MB/s eta 0:00:00
ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
%load_ext tensorboard

## 파일 로드 및 전처리

In [ ]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, Bidirectional, LSTM, BatchNormalization
from keras.initializers import Ones, Zeros
from keras.models import Model, load_model
from keras.losses import Huber
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Lambda
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import RMSE
from pytorch_lightning import Trainer
import pytorch_lightning as pl
from torch.optim import Adam
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Google Drive 경로 설정
base_path = "/content/drive/MyDrive/종설프1_5팀"
train_data_path = os.path.join(base_path, "전처리.ipynb/train_data.csv")
test_data_path = os.path.join(base_path, "전처리.ipynb/test_data.csv")
lstm_checkpoint_path = os.path.join(base_path, 'lstm_model_checkpoint.weights.h5')

# 데이터 로드
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)

# 스케일링
scaler = MinMaxScaler()
df_train['feed_pressure'] = scaler.fit_transform(df_train[['feed_pressure']])
df_test['feed_pressure'] = scaler.transform(df_test[['feed_pressure']])

# 시간 인덱스 추가
df_train['time_idx'] = range(len(df_train))
df_test['time_idx'] = range(len(df_train), len(df_train) + len(df_test))

# 학습 및 테스트 데이터 병합
df_total = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
# 시퀀스 생성 함수
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])  # 길이 sequence_length 만큼의 입력
        y.append(data[i + sequence_length])    # 목표 변수는 sequence_length 이후 값
    return np.array(X), np.array(y)

# 시퀀스 길이 설정
sequence_length = 500 #max(10, total_data_length // 10)

# 그룹 ID 설정
df_total["group"] = "series"  # 단일 시계열일 경우 고유 그룹 지정

# 피드 압력 데이터를 사용하여 시퀀스 생성
features = df_total[['feed_pressure']].values
X, y = create_sequences(features, sequence_length)
df = df_total

In [ ]:
from tensorflow.keras.initializers import Ones, Zeros
from tensorflow.keras.layers import Lambda
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Softmax
import numpy as np

class BatchAccumulateModel:
    def __init__(self, model, accumulation_steps):
        self.model = model
        self.accumulation_steps = accumulation_steps
        self.accumulation_counter = 0
        self.accumulated_grads = [tf.zeros_like(var) for var in model.trainable_variables]

    def accumulate(self, inputs, targets):
        with tf.GradientTape() as tape:
            predictions = self.model(inputs, training=True)
            loss = self.model.compiled_loss(targets, predictions)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.accumulated_grads = [accumulated_grad + grad for accumulated_grad, grad in zip(self.accumulated_grads, grads)]
        self.accumulation_counter += 1
        return loss

    def apply_accumulated_gradients(self):
        if self.accumulation_counter >= self.accumulation_steps:
            self.model.optimizer.apply_gradients(zip(self.accumulated_grads, self.model.trainable_variables))
            self.accumulated_grads = [tf.zeros_like(var) for var in self.model.trainable_variables]
            self.accumulation_counter = 0

    def train_on_batch(self, inputs, targets):
        loss = self.accumulate(inputs, targets)
        if self.accumulation_counter == self.accumulation_steps:
            self.apply_accumulated_gradients()
        return loss

In [ ]:
# 전체 데이터를 사용해 시퀀스 생성
features = df[['feed_pressure']].values
X_feed_seq, y_feed_seq = create_sequences(features, sequence_length)

# 학습 및 검증 데이터 분할 (예: 80% 학습, 20% 검증)
split_ratio = 0.8
split_index = int(len(X_feed_seq) * split_ratio)

X_feed_train = X_feed_seq[:split_index]
y_feed_train = y_feed_seq[:split_index]
X_feed_val = X_feed_seq[split_index:]
y_feed_val = y_feed_seq[split_index:]

# reshape (필요시)
X_feed_train = X_feed_train.reshape((X_feed_train.shape[0], X_feed_train.shape[1], 1))
X_feed_val = X_feed_val.reshape((X_feed_val.shape[0], X_feed_val.shape[1], 1))



In [ ]:
# 결과 확인
print(f"Train set shape: {X_feed_train.shape}, {y_feed_train.shape}")
print(f"Validation set shape: {X_feed_val.shape}, {y_feed_val.shape}")

# 함수 정의

In [ ]:
from tensorflow.keras.initializers import Ones, Zeros

class LayerNormalization(tf.keras.layers.Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
      self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:], initializer=Ones(), trainable=True)
      self.beta = self.add_weight(name='beta', shape=input_shape[-1:], initializer=Zeros(), trainable=True)
      super(LayerNormalization, self).build(input_shape)

    def call(self, x):
        mean = tf.keras.backend.mean(x, axis=-1, keepdims=True)
        std = tf.keras.backend.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
class ScaledDotProductAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, attn_dropout=0.1):
        super(ScaledDotProductAttention, self).__init__()
        self.temper = np.sqrt(d_model)
        self.dropout = Dropout(attn_dropout)
        self.softmax = tf.keras.layers.Softmax()

    def call(self, q, k, v, mask=None):
        attn = tf.keras.backend.batch_dot(q, k, axes=[2, 2]) / self.temper
        if mask is not None:
            attn += (-1e+10) * (1 - mask)
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = tf.keras.backend.batch_dot(attn, v)
        return output, attn

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, n_head, d_model, d_k, d_v, dropout, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v
        self.qs_layer = Dense(n_head * d_k, use_bias=False)
        self.ks_layer = Dense(n_head * d_k, use_bias=False)
        self.vs_layer = Dense(n_head * d_v, use_bias=False)
        self.attention = ScaledDotProductAttention(d_model, attn_dropout=dropout)
        self.w_o = Dense(d_model)
        self.layer_norm = LayerNormalization()
        self.dropout = Dropout(dropout)

    def call(self, q, k, v, mask=None):
        batch_size = tf.shape(q)[0]

        # Linear projections
        qs = self.qs_layer(q)
        ks = self.ks_layer(k)
        vs = self.vs_layer(v)

        # Split heads
        qs = tf.reshape(qs, (batch_size, -1, self.n_head, self.d_k))
        ks = tf.reshape(ks, (batch_size, -1, self.n_head, self.d_k))
        vs = tf.reshape(vs, (batch_size, -1, self.n_head, self.d_v))

        # Transpose for attention dot product
        qs = tf.transpose(qs, [0, 2, 1, 3])
        ks = tf.transpose(ks, [0, 2, 1, 3])
        vs = tf.transpose(vs, [0, 2, 1, 3])

        # Reshape for batch computation
        qs = tf.reshape(qs, (-1, tf.shape(qs)[2], self.d_k))
        ks = tf.reshape(ks, (-1, tf.shape(ks)[2], self.d_k))
        vs = tf.reshape(vs, (-1, tf.shape(vs)[2], self.d_v))

        if mask is not None:
            mask = tf.keras.backend.repeat_elements(mask, self.n_head, axis=0)

        # Scaled dot-product attention
        head, attn = self.attention(qs, ks, vs, mask)

        # Reshape back to original
        head = tf.reshape(head, (batch_size, self.n_head, -1, self.d_v))
        head = tf.transpose(head, [0, 2, 1, 3])
        head = tf.reshape(head, (batch_size, -1, self.n_head * self.d_v))

        # Final linear layer
        outputs = self.w_o(head)
        outputs = self.dropout(outputs)
        return self.layer_norm(outputs), attn

### 모델 정의

### LSTM 모델

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from keras.layers import Input, Dense, Bidirectional, LSTM, LayerNormalization, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import datetime
import os

# LSTM 모델 정의 및 생성
class LSTMOnlyModel:
    def __init__(self, maxlen, embed_size):
        self.maxlen = maxlen
        self.embed_size = embed_size

    def build_model(self):
        # 입력층 (3차원 입력)
        src_seq_input = Input(shape=(self.maxlen, 1))

        # LSTM 레이어
        lstm_out = Bidirectional(LSTM(128, return_sequences=True))(src_seq_input)
        lstm_out = LayerNormalization()(lstm_out)
        lstm_out = Bidirectional(LSTM(64, return_sequences=True))(lstm_out)
        lstm_out = LayerNormalization()(lstm_out)

        # Global pooling
        avg_pool = GlobalAveragePooling1D()(lstm_out)
        max_pool = GlobalMaxPooling1D()(lstm_out)
        conc = concatenate([avg_pool, max_pool])

        # Fully connected layer
        conc = Dense(64, activation="relu")(conc)
        output = Dense(1, activation="linear")(conc)

        # 모델 생성 및 컴파일
        self.model = Model(inputs=src_seq_input, outputs=output)
        self.model.compile(loss='mse', optimizer=Adam(learning_rate=0.001, clipnorm=1.0), metrics=['mae'])

        return self.model

# 모델 체크포인트 경로 설정
lstm_checkpoint_path = 'lstm_model_checkpoint.weights.h5'

# LSTM 모델 생성
lstm_model_instance = LSTMOnlyModel(maxlen=500, embed_size=300)
lstm_model = lstm_model_instance.build_model()

# 모델의 구조를 요약하여 출력
lstm_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 500, 1)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 500, 256)       │        133,120 │ input_layer[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 500, 256)       │            512 │ bidirectional[0][0]    │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 500, 128)       │        164,352 │ layer_normalization[0… │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 500, 128)       │            256 │ bidirectional_1[0][0]  │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 128)            │              0 │ layer_normalization_1… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d      │ (None, 128)            │              0 │ layer_normalization_1… │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 256)            │              0 │ global_average_poolin… │
│                           │                        │                │ global_max_pooling1d[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │         16,448 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │             65 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 314,753 (1.20 MB)

 Trainable params: 314,753 (1.20 MB)

 Non-trainable params: 0 (0.00 B)

## 모델 학습 및 시각화

### LSTM 모델

In [ ]:
# 가중치 로드
if os.path.exists(lstm_checkpoint_path):
    lstm_model.load_weights(lstm_checkpoint_path, skip_mismatch=True)
    print("이전 체크포인트에서 가중치를 로드했습니다.")
else:
    print("체크포인트가 존재하지 않아 새로 학습을 시작합니다.")

# 콜백 설정
checkpoint_callback = ModelCheckpoint(
    filepath=lstm_checkpoint_path.replace('.h5', '') + '.h5',
    save_weights_only=True,
    save_best_only=True,
    verbose=1
)
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,  # patience 값을 늘려서 좀 더 안정적으로 학습하게 함
    verbose=1,
    restore_best_weights=True
)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

# TensorBoard 콜백 설정
log_dir = "logs/lstm_only_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True
)

X_train, X_val = X_feed_train, X_feed_val
y_train, y_val = y_feed_train, y_feed_val


# 모델 학습
# 전체 데이터로 학습하되, 일부를 검증 데이터로 사용하도록 설정
lstm_model.fit(
    X_feed_train, y_feed_train,
    epochs=10,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard_callback, checkpoint_callback, early_stopping_callback, lr_scheduler],
    batch_size=32,  # 적절한 배치 사이즈를 설정
    verbose=2
)

체크포인트가 존재하지 않아 새로 학습을 시작합니다.
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.00094, saving model to /content/drive/MyDrive/종설프1_5팀/lstm_model_checkpoint.weights.h5
267/267 - 21s - 78ms/step - loss: 8.4466e-04 - mae: 0.0195 - val_loss: 9.3656e-04 - val_mae: 0.0207 - learning_rate: 1.2500e-04
Epoch 2/10

Epoch 2: val_loss improved from 0.00094 to 0.00085, saving model to /content/drive/MyDrive/종설프1_5팀/lstm_model_checkpoint.weights.h5
267/267 - 21s - 80ms/step - loss: 8.0724e-04 - mae: 0.0187 - val_loss: 8.5457e-04 - val_mae: 0.0193 - learning_rate: 1.2500e-04
Epoch 3/10

Epoch 3: val_loss did not improve from 0.00085
267/267 - 21s - 77ms/step - loss: 8.4540e-04 - mae: 0.0196 - val_loss: 0.0013 - val_mae: 0.0277 - learning_rate: 1.2500e-04
Epoch 4/10

Epoch 4: val_loss did not improve from 0.00085

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
267/267 - 21s - 77ms/step - loss: 8.4361e-04 - mae: 0.0195 - val_loss: 9.2444e-04 - val_mae: 0.0206 - learning_ra

## 예측

### 모델 로드

In [ ]:
# Google Drive에 저장된 체크포인트 경로
checkpoint_path = "/content/drive/MyDrive/종설프1_5팀/lstm_model_checkpoint.weights.h5"

# LSTM 모델 정의 및 생성 (모델 구조를 재생성)
lstm_model_instance = LSTMOnlyModel(maxlen=sequence_length, embed_size=300)
lstm_model = lstm_model_instance.build_model()

# 체크포인트에서 가중치 로드
if os.path.exists(checkpoint_path):
    lstm_model.load_weights(checkpoint_path)
    print("이전 체크포인트에서 가중치를 로드했습니다.")
else:
    print("체크포인트가 존재하지 않아 새로 학습해야 합니다.")

# 모델을 GPU로 이동
import tensorflow as tf
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
with tf.device(device):
    lstm_model.compile(optimizer='adam', loss='mse', metrics=['mae'])


이전 체크포인트에서 가중치를 로드했습니다.


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


### 예측 데이터 생성

In [ ]:
# 'group' 컬럼 확인 및 추가
if "group" not in df_test.columns or df_test["group"].isnull().any():
    df_test["group"] = "series"  # 기본 그룹 값 설정

# Rolling Prediction 실행 전에 데이터 유효성 확인
if len(df_test) < sequence_length:
    raise ValueError(
        f"Test dataset length ({len(df_test)}) is shorter than sequence_length ({sequence_length})."
    )

In [ ]:
def rolling_predict_batch(
    model, train_dataset, sequence_length, scaler, condition_value=17, tolerance=0.1, batch_size=10, device="/GPU:0"
):
    """
    Rolling 방식으로 한 번에 batch_size만큼 예측을 수행하며 조건 만족 시까지 반복합니다.

    Args:
        model: Keras 모델.
        train_dataset: 학습에 사용된 데이터셋 (DataFrame 형태).
        sequence_length: 롤링 윈도우의 길이.
        scaler: MinMaxScaler (스케일링 복원을 위해 필요).
        condition_value: 예측이 중단되는 조건 값 (예: 17).
        tolerance: 조건 값에 대한 허용 오차.
        batch_size: 한 번에 예측할 예측 길이.
        device: 사용할 디바이스 (default: "/GPU:0").

    Returns:
        predictions (list): 예측된 값의 리스트.
    """
    with tf.device(device):
        # Initialize with the first encoder window from the training data
        input_data = train_dataset.iloc[:sequence_length].copy()
        predictions = []

        step = 0
        while True:
            try:
                # Prepare input data for prediction
                input_sequence = input_data[['feed_pressure']].values.reshape(1, sequence_length, 1)

                # Predict the next batch of values
                batch_predictions = model.predict(input_sequence, batch_size=batch_size)

                # Restore the scaled predictions to original scale
                batch_predictions_actual = scaler.inverse_transform(batch_predictions).flatten()
                predictions.extend(batch_predictions_actual)

                print(f"Step {step}: Predicted Batch = {batch_predictions_actual}")

                # Check if any prediction in the batch meets the condition
                if any(abs(pred - condition_value) <= tolerance for pred in batch_predictions_actual):
                    print(f"Condition met: A predicted value is within tolerance of {condition_value}. Stopping prediction.")
                    break

                # Update rolling input data with the new batch of predicted values
                for pred in batch_predictions_actual:
                    new_time_idx = input_data["time_idx"].iloc[-1] + 1
                    new_row = {
                        "time_idx": new_time_idx,
                        "feed_pressure": pred,
                        "group": "series",
                    }
                    input_data = pd.concat([input_data.iloc[1:], pd.DataFrame([new_row])], ignore_index=True)

                step += 1

            except Exception as e:
                print(f"Error at step {step}: {e}")
                break

    return predictions

In [ ]:
rolling_predictions = rolling_predict_batch(
    model=lstm_model,
    train_dataset=df_train,
    sequence_length=sequence_length,
    scaler=scaler,
    condition_value=17,
    tolerance=0.1,
    batch_size=10,
    device="/GPU:0",
)

print("Final Predictions:", rolling_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 964ms/step
Step 0: Predicted Batch = [9.153663]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Step 1: Predicted Batch = [11.632228]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Step 2: Predicted Batch = [11.546283]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Step 3: Predicted Batch = [11.445642]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Step 4: Predicted Batch = [11.37965]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Step 5: Predicted Batch = [11.331464]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Step 6: Predicted Batch = [11.301253]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Step 7: Predicted Batch = [11.287225]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Step 8: Predicted Batch = [11.279804]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Step 9: Predicted Batch = [11.276355]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Step 10: Predicted Batch = [11.276446]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Step 11: Predicted Batch = [11.27968]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Step 12: Predicted Batch = [11.286227]
1/1 ━━━━━━━

In [ ]:
# 결과 시각화 함수
def plot_rolling_predictions(predicted_values, condition_value=17):
    plt.figure(figsize=(12, 6))
    plt.plot(range(len(predicted_values)), predicted_values, label="Predicted", linestyle="-", color="red")
    plt.axhline(y=condition_value, color="green", linestyle="--", label=f"Condition Value ({condition_value})")
    plt.xlabel("Prediction Steps")
    plt.ylabel("Feed Pressure (Actual Scale)")
    plt.title("Predicted Feed Pressure Until Condition Value (Rolling)")
    plt.legend()
    plt.grid(True)
    plt.show()

# Rolling Predictions 그래프 호출
plot_rolling_predictions(rolling_predictions, condition_value=17)
